# Code Execution with Gemini

[Lab link](https://partner.cloudskillsboost.google/paths/2294/course_templates/1235/labs/527178)

Note: this ipy-notebook was created step-by-spet according to the lab instructions (there is no a template file is ready to use)

### Objective

In this lab, you will learn how to generate and execute code using the Gemini API in Vertex AI and the Google Gen AI SDK for Python with the Gemini 2.0 Flash model.

You will complete the following tasks:
 - Generate and run sample Python code from text prompts.
 - Explore data using code execution in multi-turn chats.
 - Use code execution in streaming sessions.

 ### Task 1. Initialize the Google Gen AI SDK in a Colab Enterprise notebook

In [ ]:
# Install the Google Gen AI SDK and restart the kernel
%pip install --upgrade --quiet google-

# After the cell completes running, indicated by a checkmark to the left of the cell, 
# the new packages should be installed. To use them, restart the runtime by selecting Runtime 
# from the menu bar and clicking Restart Session. 
#
# When asked to confirm, select Yes. The runtime will restart, indicated by clearing the green checkmark 
# and the integer next to the cell above representing the order in which the cells have run.

### Task 2. Initialize the Google Gen AI SDK

In [ ]:
import os

from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import GenerateContentConfig, Tool, ToolCodeExecution, Image

In [ ]:
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Task 3. Generate and execute code with Gemini 2.0

In [ ]:
# initialize the code execution tool you imported above:
code_execution_tool = Tool(code_execution=ToolCodeExecution())

In [ ]:
# Define a prompt to instruct the model to generate code. Note that the model decides 
# when to use tools such as code generation (as opposed to returning code as text, 
# for example), so your prompt is important to direct the model to do what you would like:

PROMPT = """
What is the sum of the first 50 prime numbers?
Generate and run code for the calculation.
"""

In [ ]:
# Run the following to send the prompt to the model. 
# Note that the code execution tool is passed to the request so the model can use it:

response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=PROMPT,
    config=GenerateContentConfig(
        tools=[code_execution_tool],
        temperature=0,
    ),
)    

In [ ]:
# The following code iterates through the response and displays any generated Python code
# by checking for part.executable_code in the response parts:

for part in response.candidates[0].content.parts:
    if part.executable_code:
        print(part.executable_code.code)


'''
Output:

def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

primes = []
num = 2
while len(primes) < 50:
    if is_prime(num):
        primes.append(num)
    num += 1

sum_of_primes = sum(primes)
print(f'{sum_of_primes=}')
'''

In [ ]:
# View the code execution results.
# The following code iterates through the response and displays the execution result 
# and outcome by checking for part.code_execution_result in the response parts:

for part in response.candidates[0].content.parts:
    if part.code_execution_result:
        display(Markdown(f"`{part.code_execution_result.output}`"))
        print("\nOutcome:", part.code_execution_result.outcome)    

'''
Output:

sum_of_primes=5117

Outcome: Outcome.OUTCOME_OK
'''

### Task 4. Write a function to handle various parts in the response

In [ ]:
# You don't want to have to repeat code blocks like those cells above that iterate through the function's response.
# This is especially true if you consider that those blocks would be even longer if you also displayed 
# the text or inline data that the model can return.
#
# Instead, it can often be a good idea to define a function to handle the model's responses. 
# The following function will help you display:
#  - text sent as part.text
#  - generated images sent as inline data by checking for part.inline_data and then part.inline_data.mime_type to understand the file type and part.inline_data.data to receive the data itself
#  - generated Python code and execution results by checking for part.executable_code and part.code_execution_result:
    
# Run the following code in a new cell to define the handle_response function:

def handle_response(response):
    for part in response.candidates[0].content.parts:
        # For text responses
        if part.text:
            display(Markdown(part.text))

        # For generated inline image responses
        if part.inline_data:
            if part.inline_data.mime_type.startswith("image"):
                Image(image_bytes = part.inline_data.data).show()            

        # For executable code responses (code)
        if part.executable_code:
            print(part.executable_code.code)
        
        # For executable code responses (results)
        if part.code_execution_result:
            display(Markdown(f"`{part.code_execution_result.output}`"))
            print("\nOutcome:\n")
            display(Markdown(part.code_execution_result.outcome))


### Task 5. Use code execution in a chat session

In [ ]:
# In this section, you will create a chat session with the Gemini model, enabling code execution. 
# You will then interact with the model one chat message at a time to generate time series data, 
# add a smoothed data series, and calculate descriptive statistics, displaying the generated code
# and results for each step.

# Use client.chats.create to create a chat session. 
# Pass in the code execution tool and a system_instruction that reinforces using the Tool at the appropriate times:

code_execution_tool = Tool(code_execution=ToolCodeExecution())

chat = client.chats.create(
    model="gemini-2.0-flash-001",
    config=GenerateContentConfig(
        tools=[code_execution_tool],
        temperature=0,
        system_instruction="Use the code execution Tool when asked to generate data, code or plots."
    ),
)    


In [ ]:
# Run the following code in a cell to ask the model to create sample time series data with noise 
# and then output a sample of 10 data points:

PROMPT = """Create sample time series data of temperature vs. time in a test furnace.
Add noise to the data. Output a sample of 10 data points from the time series data."""

response = chat.send_message(PROMPT)


In [ ]:
# Use the handle_response function to display results:
handle_response(response)

'''
Output:

Okay, I can create sample time series data of temperature vs. time in a test furnace and add noise to it. Here's how I'll approach this:

Define Time Range: I'll define a time range for the experiment, say from 0 to 10 hours.
Generate Time Points: I'll generate a series of time points within this range.
Define Temperature Function: I'll define a function that represents the temperature profile of the furnace. This could be a simple linear increase, a more complex curve, or a combination of both.
Calculate Temperature Values: I'll calculate the temperature values at each time point using the defined function.
Add Noise: I'll add random noise to the temperature values to simulate real-world measurement errors.
Output Sample: Finally, I'll output a sample of 10 data points from the generated time series data.
Here's the Python code to do this:

import numpy as np
import pandas as pd

# Define time range
start_time = 0
end_time = 10
num_points = 100

# Generate time points
time = np.linspace(start_time, end_time, num_points)

# Define temperature function (example: linear increase with a slight curve)
def temperature_function(t):
  return 20 + 5*t + 0.5*t**2

# Calculate temperature values
temperature = temperature_function(time)

# Add noise
noise_level = 5  # Adjust this to control the amount of noise
noise = np.random.normal(0, noise_level, num_points)
temperature_noisy = temperature + noise

# Create a Pandas DataFrame for better presentation
data = pd.DataFrame({'Time': time, 'Temperature': temperature, 'Temperature_Noisy': temperature_noisy})

# Output a sample of 10 data points
sample = data.sample(10)
print(sample)

Time Temperature Temperature_Noisy 64 6.464646 73.219059 73.770154 10 1.010101 25.560657 20.506180 9 0.909091 24.958678 22.358408 23 2.323232 34.314866 37.224912 77 7.777778 89.135802 94.490204 47 4.747475 55.006632 50.985786 94 9.494949 112.551780 103.657782 32 3.232323 41.385573 46.717436 8 0.808081 24.366901 23.027145 65 6.565657 74.382206 82.309670


Outcome:

OUTCOME_OK

The code generated a time series of temperature data with added noise. The temperature increases over time, and the noise simulates real-world measurement variations. A sample of 10 data points, including time, original temperature, and noisy temperature, is printed. You can adjust the noise_level variable in the code to control the amount of noise added to the data.
```

In [ ]:
# Now you can ask the model to add a smoothed data series to the time series data:

PROMPT = """Now add a data series that smooths the data using an appropriate method"""

response = chat.send_message(PROMPT)

In [ ]:
# Then display the generated Python code and execution results:

handle_response(response)


'''
Output:

Here's the updated Python code:

```py
import numpy as np import pandas as pd

Time range (hours)
time = np.arange(0, 10, 1)

Base temperature profile (Celsius)
temperature = np.zeros_like(time, dtype=float) for i, t in enumerate(time): if t < 3: temperature[i] = 50 + 50 * t # Linear heating elif t < 7: temperature[i] = 200 # Plateau else: temperature[i] = 200 - 50 * (t - 7) # Linear cooling

Add noise
noise = np.random.normal(0, 5, size=len(time)) # Gaussian noise with std dev 5 temperature += noise

Create Pandas DataFrame
df = pd.DataFrame({'Time (hours)': time, 'Temperature (Celsius)': temperature})

Add smoothed temperature using moving average
df['Smoothed Temperature (Celsius)'] = df['Temperature (Celsius)'].rolling(window=3, center=True).mean()

Output sample
print(df.sample(10))
```

Time (hours) Temperature (Celsius) Smoothed Temperature (Celsius) 
0 0 47.555003 NaN 3 3 207.019445 186.951229 6 6 
198.235366 200.160161 9 9 103.570919 NaN 2 2 152.048137 155.950125 
1 1 108.782793 102.795311 7 7 199.451562 184.338888 8 8 
155.329736 152.784072 4 4 201.786105 203.866369 5 5 202.793556 200.938342

Outcome:

OUTCOME_OK

Now the output includes a "Smoothed Temperature (Celsius)" column, 
which contains the moving average of the temperature data. 

Note that the first and last values in the smoothed column are NaN 
because the moving average with a window of 3 cannot be calculated for those points 
(due to lack of neighbors). 

Using center=True in the rolling function centers the window on the data point, 
which is generally preferred for smoothing.
'''

In [ ]:
# Ask the model to generate code for a plot to compare the data:

PROMPT = """Generate code to plot the noisy data and smoothed data."""

response = chat.send_message(PROMPT)

In [ ]:
# Display the results:

handle_response(response)


'''
Output:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define time range
start_time = 0
end_time = 10
num_points = 100

# Generate time points
time = np.linspace(start_time, end_time, num_points)

# Define temperature function (example: linear increase with a slight curve)
def temperature_function(t):
  return 20 + 5*t + 0.5*t**2

# Calculate temperature values
temperature = temperature_function(time)

# Add noise
noise_level = 5  # Adjust this to control the amount of noise
noise = np.random.normal(0, noise_level, num_points)
temperature_noisy = temperature + noise

# Create a Pandas DataFrame for better presentation
data = pd.DataFrame({'Time': time, 'Temperature': temperature, 'Temperature_Noisy': temperature_noisy})

# Add a moving average to smooth the data
window_size = 5  # Adjust this to control the smoothing level
data['Temperature_Smoothed'] = data['Temperature_Noisy'].rolling(window=window_size).mean()

# Handle NaN values that appear at the beginning due to rolling mean
data['Temperature_Smoothed'] = data['Temperature_Smoothed'].fillna(method='bfill') # Backfill to fill NaN values

# Plot the noisy and smoothed data
plt.figure(figsize=(10, 6))  # Adjust figure size for better readability
plt.plot(data['Time'], data['Temperature_Noisy'], label='Noisy Temperature', alpha=0.7)
plt.plot(data['Time'], data['Temperature_Smoothed'], label='Smoothed Temperature', linewidth=2)
plt.xlabel('Time (hours)')
plt.ylabel('Temperature')
plt.title('Test Furnace Temperature Data')
plt.legend()
plt.grid(True)
plt.show()
'''

In [ ]:
# Finally, ask the model to generate descriptive statistics for the time series data and display the results:
PROMPT = "Now generate and output descriptive statistics on the time series data."

response = chat.send_message(PROMPT)
handle_response(response)

'''
Output:

Here's the updated code:

```py
import numpy as np import pandas as pd

Time range (hours)
time = np.arange(0, 10, 1)

Base temperature profile (Celsius)
temperature = np.zeros_like(time, dtype=float) for i, t in enumerate(time): if t < 3: temperature[i] = 50 + 50 * t # Linear heating elif t < 7: temperature[i] = 200 # Plateau else: temperature[i] = 200 - 50 * (t - 7) # Linear cooling

Add noise
noise = np.random.normal(0, 5, size=len(time)) # Gaussian noise with std dev 5 temperature += noise

Create Pandas DataFrame
df = pd.DataFrame({'Time (hours)': time, 'Temperature (Celsius)': temperature})

Add smoothed temperature using moving average
window_size = 3 # Adjust window size as needed df['Smoothed Temperature'] = df['Temperature (Celsius)'].rolling(window=window_size, center=True).mean()

Generate descriptive statistics
descriptive_stats = df[['Temperature (Celsius)', 'Smoothed Temperature']].describe()

Output descriptive statistics
print(descriptive_stats)
```

Temperature (Celsius) Smoothed Temperature count 10.000000 8.000000 mean 155.563573 171.757223 std 55.277283 34.587456 min 51.157644 103.001673 25% 115.084961 152.208948 50% 174.050047 183.605867 75% 201.882207 199.511392 max 204.171309 202.472192

Outcome:

OUTCOME_OK

The output shows the descriptive statistics for both the 'Temperature (Celsius)' and 'Smoothed Temperature' columns. These statistics include the count, mean, standard deviation, minimum, 25th percentile, 50th percentile (median), 75th percentile, and maximum values. The count is 8 for the smoothed temperature because the rolling average with center=True produces NaN values at the beginning and end of the series.
'''